# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [8]:
with open("real_estate_sales_data.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [9]:
from langchain.text_splitter import CharacterTextSplitter

In [10]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [12]:
docs = text_splitter.create_documents([real_estate_sales])

In [13]:
docs[0]

Document(page_content='以下是100条实用的比亚迪汽车销售话术，供职场新人参考：\n\n\\d+\\.\n[客户问题] 这款车的油耗怎么样？\n[销售回答] 比亚迪的电动车几乎不需要油耗，电能成本仅为传统燃油车的三分之一，经济实惠。')

In [14]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [46]:
query = "汽车性能怎么样"

In [47]:
answer_list = db.similarity_search(query)

In [48]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这款车的动力表现如何？
[销售回答] 比亚迪电动车的电动机瞬时扭矩输出，加速性能非常出色，给您带来非凡的驾驶体验。

[客户问题] 这款车的驾驶体验如何？  
[销售回答] 我们的电动车驾驶体验非常顺畅，电动机的瞬时响应让您感受到强大的动力。

[客户问题] 这款车的环保性能如何？  
[销售回答] 比亚迪电动车零排放，符合国家环保标准，是绿色出行的理想选择。

[客户问题] 这款车的外观设计有什么亮点？  
[销售回答] 我们的车型外观时尚动感，流线型设计不仅美观，还能有效降低风阻。



In [20]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [21]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [22]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000223C163D270>, search_kwargs={'k': 3})

In [23]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 这款车的动力表现如何？
[销售回答] 比亚迪电动车的电动机瞬时扭矩输出，加速性能非常出色，给您带来非凡的驾驶体验。

[客户问题] 这款车的驾驶体验如何？  
[销售回答] 我们的电动车驾驶体验非常顺畅，电动机的瞬时响应让您感受到强大的动力。

[客户问题] 这款车的环保性能如何？  
[销售回答] 比亚迪电动车零排放，符合国家环保标准，是绿色出行的理想选择。



In [24]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪车啊？")

In [25]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车的车主社区活跃吗？  
[销售回答] 我们有一个活跃的车主社区，您可以在这里交流用车经验，获取更多信息。

[客户问题] 这款车的车主社区活跃吗？  
[销售回答] 我们有一个活跃的车主社区，您可以在这里交流用车经验，获取更多信息。

[客户问题] 这款车的车主社区活跃吗？
[销售回答] 我们有一个活跃的车主社区，您可以在这里交流用车经验，获取更多信息。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [26]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [27]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车的动力表现如何？
[销售回答] 比亚迪电动车的电动机瞬时扭矩输出，加速性能非常出色，给您带来非凡的驾驶体验。

[客户问题] 这款车的驾驶体验如何？  
[销售回答] 我们的电动车驾驶体验非常顺畅，电动机的瞬时响应让您感受到强大的动力。



### 提取向量数据库中的`销售回答`

In [28]:
docs = retriever.get_relevant_documents(query)

In [29]:
docs[0].page_content

'[客户问题] 这款车的动力表现如何？\n[销售回答] 比亚迪电动车的电动机瞬时扭矩输出，加速性能非常出色，给您带来非凡的驾驶体验。'

In [30]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 这款车的动力表现如何？\n', '比亚迪电动车的电动机瞬时扭矩输出，加速性能非常出色，给您带来非凡的驾驶体验。']

In [31]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [32]:
ans

'比亚迪电动车的电动机瞬时扭矩输出，加速性能非常出色，给您带来非凡的驾驶体验。'

#### 尝试各种问题

In [33]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [34]:
query = "我想便宜一点"

print(sales(query))

[]


C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [35]:
print(sales(query, 0.75))

[]


C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


In [36]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们的起步价非常具有竞争力，您可以根据自己的预算选择合适的车型。', '我们的车型价格非常具有竞争力，且电动车享有国家和地方的补贴政策，实际购车成本更低。', '我们的内饰设计现代时尚，采用高品质材料，配备先进的智能科技，给您带来愉悦的驾乘体验。', '比亚迪提供整车八年或15万公里的保修政策，让您无后顾之忧。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [37]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [38]:
qa_chain({"query": "你们小区有100万的车子吗？"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有100万的车子吗？', 'result': '我不知道。'}

In [39]:
qa_chain({"query": "汽车油耗怎么样"})

{'query': '汽车油耗怎么样', 'result': '比亚迪的电动车几乎不需要油耗，电能成本仅为传统燃油车的三分之一，经济实惠。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [45]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [49]:
qa_chain({"query": "汽车油耗怎么样"})

{'query': '汽车油耗怎么样', 'result': '比亚迪的电动车几乎不需要油耗，电能成本仅为传统燃油车的三分之一，经济实惠。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}